<a href="https://colab.research.google.com/github/kavish-24/Konkani_Mental_Health/blob/main/DataPreparationPrudentMedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install evaluate odfpy pydub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.0/717.0 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160673 sha256=b890f2c844ddccad730833bfe0fb06ca625e6aa0319835f46a49b8a7a0055d4c
  Stored in directory: /root/.cache/pip/wheels/36/5d/63/8243a7ee78fff0f944d638fd0e66d7278888f5e2285d7346b6
Successfully built odfpy


In [ ]:
from odf import text, teletype
from odf.opendocument import OpenDocumentText
from odf.style import Style, TextProperties
from odf.text import P
import os
import re

def extract_text_from_odt(odt_path, skip_bold=True, debug=False):
    """
    Extract text from ODT file, skipping all bold text (including letters, spaces, and punctuation)
    and removing all noise (punctuation, extra spaces) from non-bold text.

    Args:
        odt_path: Path to input ODT file
        skip_bold: If True, skip all bold text
        debug: If True, print debug information

    Returns:
        List of cleaned non-bold text paragraphs
    """
    from odf.opendocument import load

    if not os.path.exists(odt_path):
        raise FileNotFoundError(f"ODT file not found: {odt_path}")

    try:
        doc = load(odt_path)
    except Exception as e:
        raise ValueError(f"Failed to load ODT file: {e}")

    bold_styles = set()

    # Check automatic styles for bold
    for style in doc.automaticstyles.getElementsByType(Style):
        style_name = style.getAttribute('name')
        for prop in style.getElementsByType(TextProperties):
            font_weight = prop.getAttribute('fontweight')
            if font_weight and 'bold' in str(font_weight).lower():
                bold_styles.add(style_name)
                if debug:
                    print(f"   Found bold style: {style_name}")

    # Check named styles for bold
    for style in doc.styles.getElementsByType(Style):
        style_name = style.getAttribute('name')
        for prop in style.getElementsByType(TextProperties):
            font_weight = prop.getAttribute('fontweight')
            if font_weight and 'bold' in str(font_weight).lower():
                bold_styles.add(style_name)
                if debug:
                    print(f"   Found bold style: {style_name}")

    print(f"\n🔍 Detected {len(bold_styles)} bold styles: {bold_styles}")

    extracted_parts = []
    skipped_count = 0
    kept_count = 0
    skipped_text_samples = []
    kept_text_samples = []

    def get_style_name(node):
        """Get style name from a node using multiple methods"""
        # Try different attribute names
        for attr_name in ['stylename', 'style-name']:
            try:
                style = node.getAttribute(attr_name)
                if style:
                    return style
            except:
                pass

        # Try namespace-aware retrieval
        try:
            style = node.getAttrNS(
                "urn:oasis:names:tc:opendocument:xmlns:text:1.0",
                "style-name"
            )
            if style:
                return style
        except:
            pass

        return None

    def is_node_bold(node):
        """Check if a node has bold styling"""
        style_name = get_style_name(node)
        if style_name and style_name in bold_styles:
            return True
        return False

    def process_node(node, parent_is_bold=False):
        """Recursively process a node and its children"""
        nonlocal skipped_count, kept_count, skipped_text_samples, kept_text_samples

        result_text = ""

        # Check if current node is bold
        current_is_bold = parent_is_bold or is_node_bold(node)

        if node.nodeType == node.TEXT_NODE:
            node_text = node.data
            if node_text.strip():
                if skip_bold and current_is_bold:
                    skipped_count += 1
                    if len(skipped_text_samples) < 5:
                        skipped_text_samples.append(node_text.strip()[:50])
                    if debug:
                        print(f"   [SKIP] TEXT_NODE (bold): {repr(node_text.strip()[:50])}")
                else:
                    result_text += node_text
                    kept_count += 1
                    if len(kept_text_samples) < 5:
                        kept_text_samples.append(node_text.strip()[:50])
                    if debug:
                        print(f"   [KEEP] TEXT_NODE: {repr(node_text.strip()[:50])}")

        elif node.nodeType == node.ELEMENT_NODE:
            # For span elements, check if they're bold
            if node.tagName == "text:span":
                span_is_bold = current_is_bold or is_node_bold(node)

                # Get all text from this span (including nested elements)
                span_full_text = teletype.extractText(node)

                if skip_bold and span_is_bold:
                    skipped_count += 1
                    if span_full_text.strip() and len(skipped_text_samples) < 5:
                        skipped_text_samples.append(span_full_text.strip()[:50])
                    if debug:
                        style_name = get_style_name(node)
                        print(f"   [SKIP] SPAN (bold - {style_name}): {repr(span_full_text[:50])}")
                    # Don't process children if parent span is bold
                    return ""
                else:
                    if debug and span_full_text.strip():
                        style_name = get_style_name(node)
                        print(f"   [KEEP] SPAN ({style_name}): {repr(span_full_text[:50])}")
                    # Process children with current bold status
                    for child in node.childNodes:
                        result_text += process_node(child, span_is_bold)
            else:
                # For other elements, process children
                for child in node.childNodes:
                    result_text += process_node(child, current_is_bold)

        return result_text

    for paragraph in doc.getElementsByType(text.P):
        # Check if paragraph itself is bold
        para_is_bold = is_node_bold(paragraph)

        if debug and para_is_bold:
            print(f"\n⚠️  Paragraph itself is BOLD - will skip all content")

        para_text = ""
        for node in paragraph.childNodes:
            para_text += process_node(node, para_is_bold)

        if para_text.strip():
            # Clean text: remove all punctuation and normalize spaces
            para_text = re.sub(r'\.{2,}', ' ', para_text)  # Replace 2+ dots with single space
            para_text = re.sub(r'[!?,;:"\'()\[\]{}\-—*]+', '', para_text)  # Remove other punctuation

            para_text = re.sub(r'\s+', ' ', para_text)  # Normalize spaces
            para_text = para_text.strip()
            if para_text:
                extracted_parts.append(para_text)

    print(f"\n📊 Extraction summary:")
    print(f"   Kept: {kept_count} elements")
    print(f"   Skipped (bold): {skipped_count} elements")
    print(f"   Extracted paragraphs: {len(extracted_parts)}")

    if skipped_count > 0:
        print(f"\n❌ Sample SKIPPED bold text:")
        for sample in skipped_text_samples:
            print(f"   • {repr(sample)}...")

    if kept_count > 0:
        print(f"\n✅ Sample KEPT non-bold text:")
        for sample in kept_text_samples:
            print(f"   • {repr(sample)}...")

    if extracted_parts:
        print(f"\n🧼 Sample CLEANED paragraphs:")
        for i, para in enumerate(extracted_parts[:3]):
            print(f"   • [{i}] {repr(para)[:100]}...")

    if skipped_count == 0 and skip_bold:
        print(f"\n⚠️  WARNING: No bold text was found to skip! Check ODT styles.")

    return extracted_parts

def create_new_odt(output_path, paragraphs):
    """
    Create a new ODT file with the given paragraphs.

    Args:
        output_path: Path to save the new ODT file
        paragraphs: List of text paragraphs to include
    """
    doc = OpenDocumentText()

    for para_text in paragraphs:
        p = P(text=para_text)
        doc.text.addElement(p)

    try:
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        doc.save(output_path)
        print(f"\n💾 New ODT file created: {output_path}")
    except Exception as e:
        raise ValueError(f"Failed to save new ODT file: {e}")

def process_directory(input_dir, output_dir, debug=False):
    """
    Process all .odt files in a directory and create cleaned files
    in the output directory.
    """

    if not os.path.exists(input_dir):
        raise FileNotFoundError(f"Input directory not found: {input_dir}")

    os.makedirs(output_dir, exist_ok=True)

    odt_files = [f for f in os.listdir(input_dir) if f.lower().endswith(".odt")]

    if not odt_files:
        print("❌ No .odt files found in input directory!")
        return

    print(f"📂 Found {len(odt_files)} ODT files to process.\n")

    for file_name in odt_files:
        input_path = os.path.join(input_dir, file_name)
        output_name = file_name.replace(".odt", "_non_bold.odt")
        output_path = os.path.join(output_dir, output_name)

        print(f"➡️ Processing: {file_name}")

        try:
            paragraphs = extract_text_from_odt(input_path, skip_bold=True, debug=debug)
            create_new_odt(output_path, paragraphs)
            print(f"✅ Saved cleaned file: {output_path}\n")

        except Exception as e:
            print(f"❌ Error processing {file_name}: {e}\n")



def main():
    # Set your folders here
    INPUT_DIR = "/content/drive/MyDrive/Anju Project (1)/Transcript/October"
    OUTPUT_DIR = "/content/drive/MyDrive/Anju Project (1)/Cleaned Transcript/Octobercleaned"
    DEBUG = False  # Set True if you want full logs

    print("="*70)
    print("PROCESSING DIRECTORY OF ODT FILES - REMOVING ALL BOLD TEXT")
    print("="*70)

    try:
        process_directory(INPUT_DIR, OUTPUT_DIR, debug=DEBUG)
        print("\n✅✅ ALL FILES PROCESSED SUCCESSFULLY ✅✅")

    except Exception as e:
        print(f"\n❌ Failed: {e}")

if __name__ == "__main__":
    main()

PROCESSING DIRECTORY OF ODT FILES - REMOVING ALL BOLD TEXT
📂 Found 8 ODT files to process.

➡️ Processing: 2 OCT PRIME.odt

🔍 Detected 70 bold styles: {'T13', 'T51', 'T58', 'P69', 'P88', 'T28', 'T87', 'T16', 'T61', 'P70', 'T7', 'T82', 'T100', 'T14', 'T112', 'T42', 'T116', 'P3', 'T75', 'T38', 'T97', 'P19', 'P5', 'T39', 'T102', 'T40', 'P45', 'P21', 'T12', 'P20', 'P10', 'T60', 'T57', 'T104', 'T30', 'P17', 'P2', 'T66', 'T93', 'T62', 'T44', 'T47', 'T25', 'T49', 'T106', 'P1', 'T37', 'T73', 'T79', 'P76', 'T109', 'T34', 'P9', 'P22', 'P4', 'T53', 'T59', 'P18', 'P71', 'T85', 'T114', 'Heading4', 'T95', 'T91', 'P8', 'T55', 'T68', 'T32', 'T36', 'T29'}

📊 Extraction summary:
   Kept: 26 elements
   Skipped (bold): 66 elements
   Extracted paragraphs: 22

❌ Sample SKIPPED bold text:
   • '2 ऑक्टोबर 2017 – प्रायम (पीके, नेने,अमर, अपर्णा)'...
   • 'अँकर – सुयश गावणेकार'...
   • '1.(प्रुडन्ट)'...
   • 'फस्ट ब्रेक'...
   • '2.(फायर)(ब्युरे)'...

✅ Sample KEPT non-bold text:
   • 'नमस्कार पळोवया प्रुडंट ख

In [ ]:
#First change the path with the ur transcript month and run this
import zipfile
from xml.etree import ElementTree as ET

file_path = "/content/drive/MyDrive/Anju Project (1)/Cleaned Transcript/Augustcleaned/7 AUG PRIME_non_bold.odt"

def read_odt(path):
    with zipfile.ZipFile(path, 'r') as z:
        xml_content = z.read("content.xml")

    root = ET.fromstring(xml_content)

    # ODT text nodes live inside <text:p> and <text:span>
    ns = {"text": "urn:oasis:names:tc:opendocument:xmlns:text:1.0"}

    paragraphs = []
    for p in root.findall(".//text:p", ns):
        text_content = ""
        for node in p.iter():
            if node.text:
                text_content += node.text
        paragraphs.append(text_content.strip())

    return "\n".join(paragraphs)

text_output = read_odt(file_path)
print(text_output)


नमस्कार पळोवया प्रुडंट खबरो
2 हजार 400 कोटींचे लुकसाण 201516चें सीएजी ऑडिट टॅक्साचे चुकीचे असेसमेंटकमी लॅव्हीचो तिजोरेक फटको मायनिंग कंपन्यांचे एक्स्ट्रा मायनिंगाचेंय सीएजी ऑडिट.
प्रुडंटान एक्स्पोज केल्ले ओपा टॅंकर स्कॅमाचो इम्पॅक्ट पीडब्ल्यूडीन केली चवकशी टॅंकर एजन्सीक 15 हजार फायन टॅन्डर टर्मिनेट पिवपाचें उदक गटारांनी सोडटना प्रुडंटान केल्लो एक्स्पोज.
हायकोर्टाची बिल्डिंग 2018त उबी 7 वर्सां हुमकळळी हायकोर्टाबरोबर पणजेचीं सगळीं कोर्टां नवे बिल्डिंगेंत सरकार स्पेसिस बिल्डिंगेचेर कोट्यांनी कड्डीता विधानसभेंत प्रस्न.
कुळ कायदो माडाचो विशय जाग्यार घालो विजय म्हण्टा दिल्ली उतरां पाळ्ळी गोवा फॉरवर्डान घेतलें पावसाळी अधिवेशनाचे अजेडाचें क्रॅडिट
हॉर्टिकल्चराचे भाज्याचें आवटलॅटांचो आकडो देंवलो भाजी सवाय क्वालिटीचें कितें विधानसभेंत प्रस्न शेतकी मंत्री म्हण्टा हॉर्टिकल्चरांत गांवठी फळभाजी घेवपाचो प्लान.
आनी
रक्षाबंधन जालें भावाक राखी बांदून भयण भावाचें नातें घटमूट करपाचो दीस शाळा कॉलेजी ऑफिसांनीय रक्षाबंधना
सीएजीचो रिपोर्ट आयलोसरकाराक 2 हजार 400 कोटींचे लुकसाणीचो असेसमेन्ट 201516 चे सीएजी ऑडिटा

In [ ]:
# First change the path with the ur transcript month and run this
import os

# path to your folder containing txt files
folder_path = "/content/drive/MyDrive/Anju Project (1)/Segment transcript/Aug/Konkani Prime News_070817"

# loop through the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".txt"):  # only .txt files
        file_path = os.path.join(folder_path, file_name)

        # open file with utf-8 encoding
        with open(file_path, 'r', encoding='utf-8') as file:
            print(f"--- {file_name} ---")
            content = file.read()
            print(content)
            print("\n")  # spacing between files


--- Konkani Prime News_070817_segment_001.txt ---
नमस्कार पोले प्रुडंट खबरो दो नजार चार्षे कोटी चे लुक्सान, दो नजार पंदर सलचे CAG ओडिट, टेक्साचे चुकी चे एसस्मेंट, कमी लेवी तो तीज़ोरेक फटको, माइनिंग कंपणे चे एक्स्ट्रा माइनिंग चे CAG ओडिट, प्रुडंट न एक्स्ट्रा


--- Konkani Prime News_070817_segment_002.txt ---
एक्सपोज केल ले आपा टैंकर स्केम आसा इंपेक्ट पीडब्ल्यूडिन केली चोकशी टैंकर एजन्सिक 15,000 फाइन टैंडर टर्मिनेट प्यूपाचे उदग गटारा नी सोटतान प्रूडन्टान केल लो एक्सपोज एक्सपोज हाइ कोटा जी बिल्डिंग 2018 दूबी साथ वर्स उम्कवली हाई कोटा बरबर पुण्जेची सगली कोटा नवे बिल्डिंग एत सरकाज स्पेसी स्पेसी स्पेसी स्पेसी स्पेसी स्पेसी स्पेसी स्पेसी स्पेसी स्पेसी स्पेसी


--- Konkani Prime News_070817_segment_003.txt ---
बिल्डिंगे चेर कोट्या निकड़ीता विदान समेध प्रस्ण कूल कायदो माडाचो विशय जागयार गलो विजय मंट दिल्ली उतरापाणी गोवा फ़र्वाडां गतले पावसाली अधिवेशाणाची अजेंडाची क्रेडिट भाजा चे आउटलेंदाचो आक्डो देवलो भाजी सवाय क्वालिटी चे गतले


--- Konkani Prime News_070817_segment_004.txt ---
विदान सब्येत प

In [ ]:
#use the prompt from the repo and do this
import re

raw = r'''
file_name,matched_text
--- Konkani Prime News_070817_segment_001.txt ---,नमस्कार पळोवया प्रुडंट खबरो 2 हजार 400 कोटींचे लुकसाण 201516चें सीएजी ऑडिट टॅक्साचे चुकीचे असेसमेंटकमी लॅव्हीचो तिजोरेक फटको मायनिंग कंपन्यांचे एक्स्ट्रा मायनिंगाचेंय सीएजी ऑडिट. प्रुडंटान एक्स्पोज केल्ले ओपा टॅंकर स्कॅमाचो
--- Konkani Prime News_070817_segment_002.txt ---,प्रुडंटान एक्स्पोज केल्ले ओपा टॅंकर स्कॅमाचो इम्पॅक्ट पीडब्ल्यूडीन केली चवकशी टॅंकर एजन्सीक 15 हजार फायन टॅन्डर टर्मिनेट पिवपाचें उदक गटारांनी सोडटना प्रुडंटान केल्लो एक्स्पोज. हायकोर्टाची बिल्डिंग 2018त उबी 7 वर्सां हुमकळळी हायकोर्टाबरोबर पणजेचीं सगळीं कोर्टां नवे बिल्डिंगेंत सरकार स्पेसिस
--- Konkani Prime News_070817_segment_003.txt ---,बिल्डिंगेंत सरकार स्पेसिस बिल्डिंगेचेर कोट्यांनी कड्डीता विधानसभेंत प्रस्न. कुळ कायदो माडाचो विशय जाग्यार घालो विजय म्हण्टा दिल्ली उतरां पाळ्ळी गोवा फॉरवर्डान घेतलें पावसाळी अधिवेशनाचे अजेडाचें क्रॅडिट हॉर्टिकल्चराचे भाज्याचें आवटलॅटांचो आकडो देंवलो भाजी सवाय क्वालिटीचें कितें
--- Konkani Prime News_070817_segment_004.txt ---,विधानसभेंत प्रस्न शेतकी मंत्री म्हण्टा हॉर्टिकल्चरांत गांवठी फळभाजी घेवपाचो प्लान. आनी रक्षाबंधन जालें भावाक राखी बांदून भयण भावाचें नातें घटमूट करपाचो दीस शाळा कॉलेजी ऑफिसांनीय रक्षाबंधना
--- Konkani Prime News_070817_segment_005.txt ---,सीएजीचो रिपोर्ट आयलोसरकाराक 2 हजार 400 कोटींचे लुकसाणीचो असेसमेन्ट 201516 चे सीएजी ऑडिटांत केला. वेगवेगळ्या 220 केशींनी वेगवेगळे तरेचे टॅक्सांचे चुकीचे असेसमेंट उणे लॅव्ही धरून तिजोरेक 2 हजार 400 कोटींचो फटको. सीएजीचो रिपोर्टविधानसभेंत आयलो. सीएजीन एक्स्ट्रा मायनिंगाचेंय ऑ़डिट केलां. कांय कंपन्यांनी खंय थारायल्ल्या मर्यादेपरस
--- Konkani Prime News_070817_segment_006.txt ---,प्रुडंटान ओपा एक्स्पोज केल्ल्या टॅंकर स्कॅमाचो इम्पॅक्ट.
--- Konkani Prime News_070817_segment_007.txt ---,विरोधी पक्ष फुडारी बाबून विशय विधानसभेंत काडिल्लो. तीन दिसांभितर पीडब्ल्युडी मंत्र्यान चवकशी रिपोर्ट दिलो पळयात प्रुडंट इम्पॅक्ट. हॉर्टिकल्चराचे भाज्याचें आवटलॅटांचो हजारांवयर पाविल्लो आकडो 931सांचेर पावला. हॉर्टिकल्चराची भाजी सवाय मेळटा खरी मात क्वालिटीचें कितें बेळगांवची भाजी सिलॅक्ट कशी करतात काय म्हूण बाबून प्रस्न काडिल्लो.
--- Konkani Prime News_070817_segment_008.txt ---,हॉर्टिकल्चर वर्साक 80 कोटींची भाजी घेता. हॉर्टिकल्चरांत सरकार गांवठी फळभाजी प्रोमोट करतलो म्हूण शेतकी मंत्री विजय उलयलो. पणजे एका बेंगळूरच्या हॉर्टिकल्चर आवटॅलाचीय चवकशी जावपीची आसा. घेवया एक ब्रेक
--- Konkani Prime News_070817_segment_009.txt ---,कॉंग्रेसीचो अध्यक्ष शांताराम नायक विरोधी पक्ष फुडारी बाबू कवळेकार सुवाळ्याक आशिल्ले. शांताराम नायकान स्मार्ट सिटी म्हणल्यार पयलीं कितें तें समजून घेवचें आनी मागीर पणजेच्या स्मार्ट सिटीचेर उलोवचे. ताणी तांचे सरकार आशिल्ल्या कर्नाटकाक पयली सल्लो दिवचो. थंयसर कर्नाटक स्मार्ट सिटीखातीर सामकें आक्रमक जEलां म्हूण पणजेचो आदलो आमदार सिद्धार्थ कुंकळ्येकार
--- Konkani Prime News_070817_segment_010.txt ---,घेवया एक ब्रेक हायकोर्टाची नवी बिल्डिंग 2018 म्हणल्यार जातली. प्रॉजॅक्ट 7 वर्सां हुमकळळा. पणजेचीं सगळीं कोर्टां थंय वतलीं म्हूण कायदो मंत्री फ्रांसिस उलयला. सरकार पाट्यार स्पेसिस बिल्डिंगेंतले कोर्टाचेर भाड्याक वर्सांक 5 कोटी कड्डीता. बिल्डिंग रेजिडॅन्सियल.
--- Konkani Prime News_070817_segment_011.txt ---,थंय कोर्ट कशें चलता काय म्हूण आलॅक्सान प्रस्न काडलो. काब्राल म्हण्टा भाड्याचे बिल्डिंगांनी कोर्टां चलोवचे परस कोर्टाची बिल्डिंग कित्याक बांदीनात. ताचेर मदींच उभो रावन पर्रीकारान रिणाचे ज्युराचीं गणितां सांगलीं. फाटलीं पाच स वर्सां जालीं आयडीसींतले एसटी एससी ओबीसींचे शंभरावयर पोस्टभरनासतना तश्शे आसात.
--- Konkani Prime News_070817_segment_012.txt ---,लोकांक घेवन वेळ मारून व्हरता म्हूण रवीन प्रस्न काडलो. मंत्री पर्रीकार म्हण्टा कित्याक भरूंक ना ते पळोवचे पडटलें. मात फुडल्या चार म्हयन्यांत रॅक्रूटमॅन्टाची नवी पॉलिसी करून कॉन्ट्रॅक्च्यूअल नोकरभरती पुराय बंद करतले म्हूण पर्रीकार उलयला.
--- Konkani Prime News_070817_segment_013.txt ---,टिसीपी मंत्री विजय सरदेसाय म्हण्टा आपल्या पार्टिन दिल्लीं म्हत्वाची उतरां अधिवेशनांत पाळ्ळी. सरकारान कुळ कायदो दुरुस्ती विधेयक आनी माडाक स्टेट ट्री करपाचे उतर पाळ्ळें म्हूण विजयान मिडियामुखार क्रॅडिट घेतलां. सोमारा अधिवेशन सोपल्या उपरांत गोवा फॉरवर्डाचे मंत्री आमदारांनी मिडियामुखार आयले. नेम आनी प्रक्रिया गुड्ड्याक मारून ओड़ीपी प्लान केल्यात. सरकार कितें एक्शन घेतलो काय म्हूण मूरगांवचो आमदार मिलींद नायकान वास्कोच्या
--- Konkani Prime News_070817_segment_014.txt ---,कब्रस्तान रस्त्याविशीं प्रस्न माडिल्लो. रस्तो 10 मीटर करचो म्हूण मागणी. मडगांवचो आमदार दिगंबर कामतीन वास्को पीडीएचेर बरोच फोग काडलो. टीसीपी मिनिस्टर म्हणटा चुकीच्यो गजाली दिसल्यार प्लानांत दुरुस्त्यो करून जाग्यार घालतले. प्रकल्पग्रस्तांक लुकसाणभरपाय दिवपाविशीं आनी भूसंपादन करपाविशीं विधेयक विधानसभेंत 20 आड 16 मतांनी पास जालें. कॉंग्रेसीन ह्या विधेयकाक हरकत घेतिल्ली.
--- Konkani Prime News_070817_segment_015.txt ---,हो कायदो टेनन्सीसारकिल्ल्यो जमनी घेतना सरकाराक प्रॉटेक्ट करतलो म्हूण मुख्यमंत्री उलयलो. ह्या बिलाउपरांत द गोवा रिक्विजिशन अँड एक्विजिशन ऑफ प्रॉपर्टी विधेयक विधानसभेन चिकित्सा समितीकडेन धाडलां. राज्य इलॅक्शन कमिशनान इलॅक्शनाच्या एक वर्स पयली पंचायत वा पालिका वॉर्ड़ांची डिलिमिटेशन
--- Konkani Prime News_070817_segment_016.txt ---,आनी रिजर्वेशन प्रक्रिया पुराय करची पडटली. हाकालागून वेंचणुकेक उबे रावतल्यांक तांच्या वॉर्डांची स्थिती कळटली. तांकां वेंचणुकेचे तयारेक वेळ मेळटलो म्हूण मुख्यमंत्री पर्रीकार उलयला. इलॅक्शन कमिशनाक मतदार यादी मात एका वर्सापयलीं तयार करूंक मेळची ना म्हूण पर्रीकार उलयलो.
--- Konkani Prime News_070817_segment_017.txt ---,स्मृती इराणीन गोंयच्या कुणबी साड्येचो राष्ट्रीय स्थरार एक वेगळेच पद्धतीन भोवमान आनी ब्रॅन्डिंग दिले. अहमदाबादांत हॅन्डलूम डेचे कार्यवाळीक ती गोंयची कुणबी साडी न्हेशिल्ली. तिणे स्वताजावन गोंयच्या कुणबी साडयेचेखाशेलेपणाचो उल्लेखूय केलो. हातमागाचे कपडे वापरून विणकारांक उर्बा दिवची म्हूण तिणे उलो माल्लो. अंदूंचो तिरसो हॅन्ड्लूम डे.
--- Konkani Prime News_070817_segment_018.txt ---,पॅरिश युथ नुवेन पांचवी नुवें पंचायत कप 2017 फुटबॉल सर्त जिकली. फायनलींत तांणी फातोड्डेच्या डॉन बॉस्को ओरेटरीक टायब्रेकरार 54 गोलान मारले. फायनल नुवेंच्या जिजस मॅरी जोसेफ चर्चीच्या ग्रावंडार जाली. इनामा दिवपाच्या सुवाळ्याक टीसीपी मंत्री विजय आशिल्लो. आमदार बाबाशान सरपंच आदले पंच सरपंचांचो भौमानूय जालो
--- Konkani Prime News_070817_segment_019.txt ---,केन्ना झगडप केन्ना मारप आनी केन्ना एकमेकाक जीव सोडप एक वेगळेच नाते भाव–भयणीचें. आज रक्षाबंधन. भावाक राखी बांदून भयण भावाचें नातें घटमूट करपाचो हो दीस. संसारांत झंय भयण भाव एकठांय आसात थंय आज हो सुवाळो. एकठांय नासल्यार राखयो पोस्टान आनी व्हॉटसॅपार. शाळा कॉलेजी ऑफिसांनीय रक्षाबंधना जातात. पळयात रक्षाबंधन.
--- Konkani Prime News_070817_segment_020.txt ---,हें बुलॅटिन हांगांच सोंपता ह्यो खबरो तुमकां प्रुडंट वॉट्सॅप ट्विटर फेसबूक आनी प्रुडंट व्हॅबसायटीचेर लायव्ह मेळटात प्रुडंटाक फॉलो करात नमस्कार'''

# Pattern for both cases:
pattern = r"""
    (?:---\s*)?                # optional leading ---
    ([\w .-]+?\.txt)           # filename ending in .txt
    (?:\s*---)?,?              # optional --- or comma after filename
    ["“]?                      # optional opening quote
    (.+?)                      # the text content
    ["”]?                      # optional closing quote
    (?=\n|$)                   # end at newline
"""

matches = re.findall(pattern, raw, flags=re.DOTALL | re.VERBOSE)

print("{")
for file, text in matches:
    text = text.strip().replace('"', '\\"')

    # Remove header if file contains "file_name"
    if "file_name" in file.lower():
        continue

    print(f'    "{file}": "{text}",')
print("}")


{
    "Konkani Prime News_070817_segment_001.txt": "नमस्कार पळोवया प्रुडंट खबरो 2 हजार 400 कोटींचे लुकसाण 201516चें सीएजी ऑडिट टॅक्साचे चुकीचे असेसमेंटकमी लॅव्हीचो तिजोरेक फटको मायनिंग कंपन्यांचे एक्स्ट्रा मायनिंगाचेंय सीएजी ऑडिट. प्रुडंटान एक्स्पोज केल्ले ओपा टॅंकर स्कॅमाचो",
    "Konkani Prime News_070817_segment_002.txt": "प्रुडंटान एक्स्पोज केल्ले ओपा टॅंकर स्कॅमाचो इम्पॅक्ट पीडब्ल्यूडीन केली चवकशी टॅंकर एजन्सीक 15 हजार फायन टॅन्डर टर्मिनेट पिवपाचें उदक गटारांनी सोडटना प्रुडंटान केल्लो एक्स्पोज. हायकोर्टाची बिल्डिंग 2018त उबी 7 वर्सां हुमकळळी हायकोर्टाबरोबर पणजेचीं सगळीं कोर्टां नवे बिल्डिंगेंत सरकार स्पेसिस",
    "Konkani Prime News_070817_segment_003.txt": "बिल्डिंगेंत सरकार स्पेसिस बिल्डिंगेचेर कोट्यांनी कड्डीता विधानसभेंत प्रस्न. कुळ कायदो माडाचो विशय जाग्यार घालो विजय म्हण्टा दिल्ली उतरां पाळ्ळी गोवा फॉरवर्डान घेतलें पावसाळी अधिवेशनाचे अजेडाचें क्रॅडिट हॉर्टिकल्चराचे भाज्याचें आवटलॅटांचो आकडो देंवलो भाजी सवाय क्वालिटीचें कितें",
    "Konkani Prime News_070817_segment_004.txt":

In [ ]:
#put the out of the above code and change the path
import os

# Directory where files will be created
output_dir = "/content/drive/MyDrive/Anju Project (1)/training/August/Konkani Prime News_070817"

# Create folder if not exists
os.makedirs(output_dir, exist_ok=True)

# Mapping of file_name → matched_text
segments ={
    "Konkani Prime News_070817_segment_001.txt": "नमस्कार पळोवया प्रुडंट खबरो 2 हजार 400 कोटींचे लुकसाण 201516चें सीएजी ऑडिट टॅक्साचे चुकीचे असेसमेंटकमी लॅव्हीचो तिजोरेक फटको मायनिंग कंपन्यांचे एक्स्ट्रा मायनिंगाचेंय सीएजी ऑडिट. प्रुडंटान एक्स्पोज केल्ले ओपा टॅंकर स्कॅमाचो",
    "Konkani Prime News_070817_segment_002.txt": "प्रुडंटान एक्स्पोज केल्ले ओपा टॅंकर स्कॅमाचो इम्पॅक्ट पीडब्ल्यूडीन केली चवकशी टॅंकर एजन्सीक 15 हजार फायन टॅन्डर टर्मिनेट पिवपाचें उदक गटारांनी सोडटना प्रुडंटान केल्लो एक्स्पोज. हायकोर्टाची बिल्डिंग 2018त उबी 7 वर्सां हुमकळळी हायकोर्टाबरोबर पणजेचीं सगळीं कोर्टां नवे बिल्डिंगेंत सरकार स्पेसिस",
    "Konkani Prime News_070817_segment_003.txt": "बिल्डिंगेंत सरकार स्पेसिस बिल्डिंगेचेर कोट्यांनी कड्डीता विधानसभेंत प्रस्न. कुळ कायदो माडाचो विशय जाग्यार घालो विजय म्हण्टा दिल्ली उतरां पाळ्ळी गोवा फॉरवर्डान घेतलें पावसाळी अधिवेशनाचे अजेडाचें क्रॅडिट हॉर्टिकल्चराचे भाज्याचें आवटलॅटांचो आकडो देंवलो भाजी सवाय क्वालिटीचें कितें",
    "Konkani Prime News_070817_segment_004.txt": "विधानसभेंत प्रस्न शेतकी मंत्री म्हण्टा हॉर्टिकल्चरांत गांवठी फळभाजी घेवपाचो प्लान. आनी रक्षाबंधन जालें भावाक राखी बांदून भयण भावाचें नातें घटमूट करपाचो दीस शाळा कॉलेजी ऑफिसांनीय रक्षाबंधना",
    "Konkani Prime News_070817_segment_005.txt": "सीएजीचो रिपोर्ट आयलोसरकाराक 2 हजार 400 कोटींचे लुकसाणीचो असेसमेन्ट 201516 चे सीएजी ऑडिटांत केला. वेगवेगळ्या 220 केशींनी वेगवेगळे तरेचे टॅक्सांचे चुकीचे असेसमेंट उणे लॅव्ही धरून तिजोरेक 2 हजार 400 कोटींचो फटको. सीएजीचो रिपोर्टविधानसभेंत आयलो. सीएजीन एक्स्ट्रा मायनिंगाचेंय ऑ़डिट केलां. कांय कंपन्यांनी खंय थारायल्ल्या मर्यादेपरस",
    "Konkani Prime News_070817_segment_006.txt": "प्रुडंटान ओपा एक्स्पोज केल्ल्या टॅंकर स्कॅमाचो इम्पॅक्ट.",
    "Konkani Prime News_070817_segment_007.txt": "विरोधी पक्ष फुडारी बाबून विशय विधानसभेंत काडिल्लो. तीन दिसांभितर पीडब्ल्युडी मंत्र्यान चवकशी रिपोर्ट दिलो पळयात प्रुडंट इम्पॅक्ट. हॉर्टिकल्चराचे भाज्याचें आवटलॅटांचो हजारांवयर पाविल्लो आकडो 931सांचेर पावला. हॉर्टिकल्चराची भाजी सवाय मेळटा खरी मात क्वालिटीचें कितें बेळगांवची भाजी सिलॅक्ट कशी करतात काय म्हूण बाबून प्रस्न काडिल्लो.",
    "Konkani Prime News_070817_segment_008.txt": "हॉर्टिकल्चर वर्साक 80 कोटींची भाजी घेता. हॉर्टिकल्चरांत सरकार गांवठी फळभाजी प्रोमोट करतलो म्हूण शेतकी मंत्री विजय उलयलो. पणजे एका बेंगळूरच्या हॉर्टिकल्चर आवटॅलाचीय चवकशी जावपीची आसा. घेवया एक ब्रेक",
    "Konkani Prime News_070817_segment_009.txt": "कॉंग्रेसीचो अध्यक्ष शांताराम नायक विरोधी पक्ष फुडारी बाबू कवळेकार सुवाळ्याक आशिल्ले. शांताराम नायकान स्मार्ट सिटी म्हणल्यार पयलीं कितें तें समजून घेवचें आनी मागीर पणजेच्या स्मार्ट सिटीचेर उलोवचे. ताणी तांचे सरकार आशिल्ल्या कर्नाटकाक पयली सल्लो दिवचो. थंयसर कर्नाटक स्मार्ट सिटीखातीर सामकें आक्रमक जEलां म्हूण पणजेचो आदलो आमदार सिद्धार्थ कुंकळ्येकार",
    "Konkani Prime News_070817_segment_010.txt": "घेवया एक ब्रेक हायकोर्टाची नवी बिल्डिंग 2018 म्हणल्यार जातली. प्रॉजॅक्ट 7 वर्सां हुमकळळा. पणजेचीं सगळीं कोर्टां थंय वतलीं म्हूण कायदो मंत्री फ्रांसिस उलयला. सरकार पाट्यार स्पेसिस बिल्डिंगेंतले कोर्टाचेर भाड्याक वर्सांक 5 कोटी कड्डीता. बिल्डिंग रेजिडॅन्सियल.",
    "Konkani Prime News_070817_segment_011.txt": "थंय कोर्ट कशें चलता काय म्हूण आलॅक्सान प्रस्न काडलो. काब्राल म्हण्टा भाड्याचे बिल्डिंगांनी कोर्टां चलोवचे परस कोर्टाची बिल्डिंग कित्याक बांदीनात. ताचेर मदींच उभो रावन पर्रीकारान रिणाचे ज्युराचीं गणितां सांगलीं. फाटलीं पाच स वर्सां जालीं आयडीसींतले एसटी एससी ओबीसींचे शंभरावयर पोस्टभरनासतना तश्शे आसात.",
    "Konkani Prime News_070817_segment_012.txt": "लोकांक घेवन वेळ मारून व्हरता म्हूण रवीन प्रस्न काडलो. मंत्री पर्रीकार म्हण्टा कित्याक भरूंक ना ते पळोवचे पडटलें. मात फुडल्या चार म्हयन्यांत रॅक्रूटमॅन्टाची नवी पॉलिसी करून कॉन्ट्रॅक्च्यूअल नोकरभरती पुराय बंद करतले म्हूण पर्रीकार उलयला.",
    "Konkani Prime News_070817_segment_013.txt": "टिसीपी मंत्री विजय सरदेसाय म्हण्टा आपल्या पार्टिन दिल्लीं म्हत्वाची उतरां अधिवेशनांत पाळ्ळी. सरकारान कुळ कायदो दुरुस्ती विधेयक आनी माडाक स्टेट ट्री करपाचे उतर पाळ्ळें म्हूण विजयान मिडियामुखार क्रॅडिट घेतलां. सोमारा अधिवेशन सोपल्या उपरांत गोवा फॉरवर्डाचे मंत्री आमदारांनी मिडियामुखार आयले. नेम आनी प्रक्रिया गुड्ड्याक मारून ओड़ीपी प्लान केल्यात. सरकार कितें एक्शन घेतलो काय म्हूण मूरगांवचो आमदार मिलींद नायकान वास्कोच्या",
    "Konkani Prime News_070817_segment_014.txt": "कब्रस्तान रस्त्याविशीं प्रस्न माडिल्लो. रस्तो 10 मीटर करचो म्हूण मागणी. मडगांवचो आमदार दिगंबर कामतीन वास्को पीडीएचेर बरोच फोग काडलो. टीसीपी मिनिस्टर म्हणटा चुकीच्यो गजाली दिसल्यार प्लानांत दुरुस्त्यो करून जाग्यार घालतले. प्रकल्पग्रस्तांक लुकसाणभरपाय दिवपाविशीं आनी भूसंपादन करपाविशीं विधेयक विधानसभेंत 20 आड 16 मतांनी पास जालें. कॉंग्रेसीन ह्या विधेयकाक हरकत घेतिल्ली.",
    "Konkani Prime News_070817_segment_015.txt": "हो कायदो टेनन्सीसारकिल्ल्यो जमनी घेतना सरकाराक प्रॉटेक्ट करतलो म्हूण मुख्यमंत्री उलयलो. ह्या बिलाउपरांत द गोवा रिक्विजिशन अँड एक्विजिशन ऑफ प्रॉपर्टी विधेयक विधानसभेन चिकित्सा समितीकडेन धाडलां. राज्य इलॅक्शन कमिशनान इलॅक्शनाच्या एक वर्स पयली पंचायत वा पालिका वॉर्ड़ांची डिलिमिटेशन",
    "Konkani Prime News_070817_segment_016.txt": "आनी रिजर्वेशन प्रक्रिया पुराय करची पडटली. हाकालागून वेंचणुकेक उबे रावतल्यांक तांच्या वॉर्डांची स्थिती कळटली. तांकां वेंचणुकेचे तयारेक वेळ मेळटलो म्हूण मुख्यमंत्री पर्रीकार उलयला. इलॅक्शन कमिशनाक मतदार यादी मात एका वर्सापयलीं तयार करूंक मेळची ना म्हूण पर्रीकार उलयलो.",
    "Konkani Prime News_070817_segment_017.txt": "स्मृती इराणीन गोंयच्या कुणबी साड्येचो राष्ट्रीय स्थरार एक वेगळेच पद्धतीन भोवमान आनी ब्रॅन्डिंग दिले. अहमदाबादांत हॅन्डलूम डेचे कार्यवाळीक ती गोंयची कुणबी साडी न्हेशिल्ली. तिणे स्वताजावन गोंयच्या कुणबी साडयेचेखाशेलेपणाचो उल्लेखूय केलो. हातमागाचे कपडे वापरून विणकारांक उर्बा दिवची म्हूण तिणे उलो माल्लो. अंदूंचो तिरसो हॅन्ड्लूम डे.",
    "Konkani Prime News_070817_segment_018.txt": "पॅरिश युथ नुवेन पांचवी नुवें पंचायत कप 2017 फुटबॉल सर्त जिकली. फायनलींत तांणी फातोड्डेच्या डॉन बॉस्को ओरेटरीक टायब्रेकरार 54 गोलान मारले. फायनल नुवेंच्या जिजस मॅरी जोसेफ चर्चीच्या ग्रावंडार जाली. इनामा दिवपाच्या सुवाळ्याक टीसीपी मंत्री विजय आशिल्लो. आमदार बाबाशान सरपंच आदले पंच सरपंचांचो भौमानूय जालो",
    "Konkani Prime News_070817_segment_019.txt": "केन्ना झगडप केन्ना मारप आनी केन्ना एकमेकाक जीव सोडप एक वेगळेच नाते भाव–भयणीचें. आज रक्षाबंधन. भावाक राखी बांदून भयण भावाचें नातें घटमूट करपाचो हो दीस. संसारांत झंय भयण भाव एकठांय आसात थंय आज हो सुवाळो. एकठांय नासल्यार राखयो पोस्टान आनी व्हॉटसॅपार. शाळा कॉलेजी ऑफिसांनीय रक्षाबंधना जातात. पळयात रक्षाबंधन.",
    "Konkani Prime News_070817_segment_020.txt": "हें बुलॅटिन हांगांच सोंपता ह्यो खबरो तुमकां प्रुडंट वॉट्सॅप ट्विटर फेसबूक आनी प्रुडंट व्हॅबसायटीचेर लायव्ह मेळटात प्रुडंटाक फॉलो करात नमस्कार",
}
for file_name, text in segments.items():
    file_path = os.path.join(output_dir, file_name)

    with open(file_path, "w", encoding="utf-8") as f:
        f.write(text)

print(f"✅ Created {len(segments)} files in folder: {output_dir}")


✅ Created 20 files in folder: /content/drive/MyDrive/Anju Project (1)/training/August/Konkani Prime News_070817


In [1]:
!pip install python-docx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.4 MB/s eta 0:00:00


In [33]:
# ==========================================
# GENERATE STRICT-MATCHER PROMPT FOR GEMINI
# Supports TXT, DOCX, ODT transcripts
# ==========================================

import zipfile
from xml.etree import ElementTree as ET
import os

# -----------------------------
# UNIVERSAL TRANSCRIPT READER
# -----------------------------

def read_txt(path):
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

def read_docx(path):
    from docx import Document
    doc = Document(path)
    return "\n".join([p.text for p in doc.paragraphs])

def read_odt(path):
    with zipfile.ZipFile(path, 'r') as z:
        xml_content = z.read("content.xml")

    root = ET.fromstring(xml_content)
    ns = {"text": "urn:oasis:names:tc:opendocument:xmlns:text:1.0"}

    paragraphs = []
    for p in root.findall(".//text:p", ns):
        collected = ""
        for node in p.iter():
            if node.text:
                collected += node.text
        paragraphs.append(collected.strip())

    return "\n".join(paragraphs)

def load_transcript(path):
    p = path.lower()
    if p.endswith(".txt"):
        return read_txt(path)
    if p.endswith(".docx"):
        return read_docx(path)
    if p.endswith(".odt"):
        return read_odt(path)
    raise ValueError("Unsupported file type!")

# -----------------------------
# 1. READ FULL TRANSCRIPT
# -----------------------------

transcript_path = "/content/drive/MyDrive/Anju Project (1)/HODTRANSCRIPT/लोखण पत्रे पिलेस्टीक.docx"
full_transcript = load_transcript(transcript_path)

# -----------------------------
# 2. READ SEGMENT TXT FILES
# -----------------------------

segments_folder = "/content/drive/MyDrive/Anju Project (1)/Segment transcript/HOD/Konkani लोखण पत्रे पिलेस्टीक"
segments = {}

for file_name in os.listdir(segments_folder):
    if file_name.endswith(".txt"):
        with open(os.path.join(segments_folder, file_name), "r", encoding="utf-8") as f:
            segments[file_name] = f.read().strip()

# -----------------------------
# 3. PRINT READY PROMPT
# -----------------------------

print("You are a strict text-matcher. Do not guess or compensate.\n")
print("Goal:")
print("Match each small segment to the exact matching text in the large transcript.")
print("If a segment does NOT clearly exist in the transcript, ignore it completely — do not try to estimate or improvise a match.\n")

print("Output Format:")
print("Return ONLY a CSV table (so I can save it as Excel) with columns:\n")
print("file_name, matched_text\n")

print("Rules:")
print("• Only include rows where there is clear, strong text match.")
print("• If no clear match exists, SKIP that segment (do not mention it).")
print("• Do not add explanations.")
print("• Do not add extra text — only plain CSV.\n")

print("Segments:")
for fn, seg in segments.items():
    print(f"\n--- {fn} ---\n{seg}")

print("\n\nFull Transcript:\n")
print(full_transcript)


You are a strict text-matcher. Do not guess or compensate.

Goal:
Match each small segment to the exact matching text in the large transcript.
If a segment does NOT clearly exist in the transcript, ignore it completely — do not try to estimate or improvise a match.

Output Format:
Return ONLY a CSV table (so I can save it as Excel) with columns:

file_name, matched_text

Rules:
• Only include rows where there is clear, strong text match.
• If no clear match exists, SKIP that segment (do not mention it).
• Do not add explanations.
• Do not add extra text — only plain CSV.

Segments:

--- Konkani लोखण पत्रे पिलेस्टीक_segment_001.txt ---
लोकन, पत्रे, प्लास्टिक, बाई पन्ने, लोकन, पत्रे, प्लास्टिक, बाटली भी असा, कैसी दिली? लोकन, पत्रे, प्लास्टिक, बाई पन्ने, लोकन, पत्रे, प्लास्टिक, बाई पन्ने, लोकन, पत्रे, प्लास्टिक, बाई पन्ने, लोकन, पत्रे, प्लास्टिक, बाई पन्ने, लोकन, पत्रे, प्लास्टिक, बाई पन्ने, लोकन, पत्रे, प्लास्टिक, बाई पन्ने, �

--- Konkani लोखण पत्रे पिलेस्टीक_segment_002.txt ---
मकाईं भ

In [34]:
import os
import re

# ===============================================
# 1. PASTE GEMINI OUTPUT HERE EXACTLY AS RECEIVED
# ===============================================

raw_output = """
file_name,matched_text
--- Konkani लोखण पत्रे पिलेस्टीक_segment_001.txt,बाय पोन्ने लोखण, पत्रे, पिलेस्टीक, बाटली भी आसा? कशी दिली? लोखण चार रुपये किलो, पिलेस्टीक स रुपय किलो. दस रुपय? ना ना बाय
--- Konkani लोखण पत्रे पिलेस्टीक_segment_002.txt,आमकाय भी कितें कमोवंक जाय. ना..ना...बाय ना...ना.. लोखण, पत्रे पिलेस्टीकSS (माचयेर कोयरा आयदणां आसात. तातुंत सामान सोदता) येSS म्हाका चोर कित्याक म्हणलो रे? हांव कितें चोर? हांये कितें चोल्ला तुजें? आं? तुमीं हिंगा कचरापेटीन, कचरा उडयता,
--- Konkani लोखण पत्रे पिलेस्टीक_segment_003.txt,तेतली खराब पत्रे पिलेस्टीक हांव काडटा आनी इकून पोट भरता आनी तूं म्हाका चोर म्हणटा? ये SS शाण्या चड शाणपट्टी दाखय नाका. त्या... त्या कुत्त्याक म्हुजेर कित्याक सोडटा आं? दवा कोण दितलो; तुजो बाप?
--- Konkani लोखण पत्रे पिलेस्टीक_segment_004.txt,येSS बापाक वयर काडनाका? मागीर तूं म्हाका खालिपिली कित्याक पिडटा? भायर तुमची कटकटी आनी घारा मुझो बेवडा बाप वाट लगाता. आमी कितें करपाचें?
--- Konkani लोखण पत्रे पिलेस्टीक_segment_005.txt,चोय..चोय (फाट दाखयत) तेणे मारून मारून म्हुजी फाट पत्रेमाफीक केल्या. चोय, चोय हिंगा चोय (पांय, कोपर, मान दाखयता) चोयलें. सा S S ला हिंगा खाने-पिने का वांदा आनी तुमी
--- Konkani लोखण पत्रे पिलेस्टीक_segment_006.txt,सा S S ला हिंगा खाने-पिने का वांदा आनी तुमी करूंक दिना काम-धंदा. बोल आमी करपाचें तरी कितें? ये s s जास्ती मिजास दाखोय नाका.
--- Konkani लोखण पत्रे पिलेस्टीक_segment_007.txt,ये s s जास्ती मिजास दाखोय नाका. सगलो नशिबाचो खेल. तुजे नशीब बरें म्हूण भगवान तुका व्हड घर दिला, कमानेवाला अप्पा दिला, खाना पकानेवाली अम्मा दिला
--- Konkani लोखण पत्रे पिलेस्टीक_segment_008.txt,तुजे नशीब बरें म्हूण भगवान तुका व्हड घर दिला, कमानेवाला अप्पा दिला, खाना पकानेवाली अम्मा दिला म्हूण
--- Konkani लोखण पत्रे पिलेस्टीक_segment_009.txt,खाना पकानेवाली अम्मा दिला म्हूण तुज्या जिंदगीची चांदी आनी आमची किडे मकौडेची जीण.
--- Konkani लोखण पत्रे पिलेस्टीक_segment_010.txt,खाना पकानेवाली अम्मा दिला म्हूण तुज्या जिंदगीची चांदी आनी आमची किडे मकौडेची जीण. हांव सकाळी खाली पेट. फाटीर साक मारून गली गलीन कचरापेटी सोदीत भोंवता. आनी तूं पोटभर नाश्ता करून
--- Konkani लोखण पत्रे पिलेस्टीक_segment_011.txt,फाटीर बॅग मारून अप्पा वांगडा इस्कूटरीर बसून ग्रोईंग ग्रो S S ईंग करीत इस्कुलान वयता. तू हिरोमाफीक बरे बरे कपडे घालता आनी हांव तुमी दिल्ले फटे, डिला, पोन्ने कपडे घालता. तू बॅट, बॉल घेवन मैदानार दोस्तावांगडा खेळटा आनी हांव कोनशाक बसून तुमचो खेल पळयता. तुमी जाय तेन्ना टि.वी चोयता
--- Konkani लोखण पत्रे पिलेस्टीक_segment_012.txt,आनी हांव खिडकीतल्यान टि.वी चोयल्यार पात्राम बायल, भुरगीं म्हाका धांवडायता. तुमचेभाशेन म्हाका चोर म्हणटात.
--- Konkani लोखण पत्रे पिलेस्टीक_segment_013.txt,तुमचेभाशेन म्हाका चोर म्हणटात. खरेंच हांव चोर? काय तुमी म्हाका चोर करूंक सोदतात? ना ना. तुमी सफल जावची ना. हांव व्हडलो
--- Konkani लोखण पत्रे पिलेस्टीक_segment_014.txt,हांव व्हडलो जालो म्हणटकीच अस्लमभाय सारको कबाडिवाला जातलो. खूब पयशे कमयतलो. पूण हाय भगवान (कपलार हात मारीत) कमयतलो तरी कशो.
--- Konkani लोखण पत्रे पिलेस्टीक_segment_015.txt,दिसभर भोंवन भोंवन दोन पयशे केल्यार, घरा तो अप्पा गिधड कसो म्हजेर झोंपाय मारता, पयशे ओडून घेता आनी दारूर उडयता. आमची जिंदगी ही ‘बिना पानी का कुआं’. अस्लमबभायन म्हाका सांगला,
--- Konkani लोखण पत्रे पिलेस्टीक_segment_016.txt,राव, थोडे दिस राव. मात्सो व्हडलो जाव दी. सब ठिकाने लागतले. तेचे भी बचपन म्हुजेभशेन आसले... म्हुणच हांव निराश जायना.
--- Konkani लोखण पत्रे पिलेस्टीक_segment_017.txt,येयलो दिस सपनां चोयत सारता. चल हॅट... लोखण पत्रे...."""

# =================================================
# 2. OUTPUT FOLDER
# =================================================
output_dir = "/content/drive/MyDrive/Anju Project (1)/training/HOD/Konkani लोखण पत्रे पिलेस्टीक"
os.makedirs(output_dir, exist_ok=True)

# =================================================
# 3. SAFER REGEX
# =================================================
pattern = r"---\s*(.*?)\s*,(.*)"

matches = re.findall(pattern, raw_output)

# =================================================
# 4. WRITE FILES
# =================================================
for file_name, text in matches:
    file_path = os.path.join(output_dir, file_name)
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(text.strip())

print(f"✅ Created {len(matches)} files in folder: {output_dir}")


✅ Created 17 files in folder: /content/drive/MyDrive/Anju Project (1)/training/HOD/Konkani लोखण पत्रे पिलेस्टीक
